<a href="https://colab.research.google.com/github/sha-linimoorthy/tubeTalk_Summary/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube pydub torch transformers accelerate

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import pytube
from pydub import AudioSegment
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def download_and_convert_audio(video_url, output_directory="."):
    try:
        # Create the output directory if it doesn't exist
        yt = pytube.YouTube(video_url)
        audio_stream = yt.streams.filter(only_audio=True).first()
        audio_file_path = audio_stream.download(output_directory)

        # Convert the audio to MP3 format
        audio = AudioSegment.from_file(audio_file_path, format="mp4")
        mp3_output_path = f"{output_directory}/audio.mp3"
        audio.export(mp3_output_path, format="mp3")

        print("Audio downloaded and saved as audio.mp3 successfully.")
    except Exception as e:
        print(f"Failed to download and convert audio: {str(e)}")

if __name__ == "__main__":
    video_url = "https://youtu.be/ky5ZB-mqZKM?si=czI_FovoXTosFqe8"
    download_and_convert_audio(video_url)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "distil-whisper/distil-medium.en"
!pip install accelerate
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)

# Pass the path to your local audio file
audio_file_path = "/content/audio.mp3"

# Read the audio file and pass it to the pipeline
result = pipe(audio_file_path)
print(result["text"])


In [ ]:
text = result["text"]
stopWords = set(stopwords.words("english"))
words = word_tokenize(text)
freqTable = dict()
for word in words:
	word = word.lower()
	if word in stopWords:
		continue
	if word in freqTable:
		freqTable[word] += 1
	else:
		freqTable[word] = 1
sentences = sent_tokenize(text)
sentenceValue = dict()

for sentence in sentences:
	for word, freq in freqTable.items():
		if word in sentence.lower():
			if sentence in sentenceValue:
				sentenceValue[sentence] += freq
			else:
				sentenceValue[sentence] = freq

sumValues = 0
for sentence in sentenceValue:
	sumValues += sentenceValue[sentence]
average = int(sumValues / len(sentenceValue))
summary = ''
for sentence in sentences:
	if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
		summary += " " + sentence
print(summary)


 She's an Instagram model from Barcelona who's been going viral, but she's also just like totally chill and down to earth. She's into fitness and video games and yeah if you're wondering she does have additional photos on our subscription tier which brings in about 10 grand per month. Before you sign up though there's one thing I need to disclose and it really shouldn't matter unless you're some kind of big it technophobic not see but she's not a biological female.


In [ ]:
import tkinter as tk
from tkinter import messagebox

# Download and convert audio function
def download_and_convert_audio(video_url, output_directory="."):
    try:
        # Create the output directory if it doesn't exist
        yt = YouTube(video_url)
        audio_stream = yt.streams.filter(only_audio=True).first()
        audio_file_path = audio_stream.download(output_directory)

        # Convert the audio to MP3 format
        audio = AudioSegment.from_file(audio_file_path, format="mp4")
        mp3_output_path = f"{output_directory}/audio.mp3"
        audio.export(mp3_output_path, format="mp3")

        messagebox.showinfo("Success", "Audio downloaded and saved as audio.mp3 successfully.")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to download and convert audio: {str(e)}")

# Transcription and summarization function
def transcribe_and_summarize(audio_file_path):
    result = pipe(audio_file_path)
    text = result["text"]
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text)
    freqTable = dict()

    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    sentences = sent_tokenize(text)
    sentenceValue = dict()

    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq

    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]
    average = int(sumValues / len(sentenceValue))
    summary = ''

    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
            summary += " " + sentence

    return summary

# GUI setup
def process_video():
    video_url = url_entry.get()
    download_and_convert_audio(video_url, output_directory="/content")
    audio_file_path = "/content/audio.mp3"
    summary_text = transcribe_and_summarize(audio_file_path)
    summary_textbox.delete("1.0", tk.END)
    summary_textbox.insert(tk.END, summary_text)

# Main GUI window
root = tk.Tk()
root.title("YouTube Audio Summarizer")

# UI components
url_label = tk.Label(root, text="Enter YouTube URL:")
url_label.pack()

url_entry = tk.Entry(root, width=50)
url_entry.pack()

process_button = tk.Button(root, text="Process Video", command=process_video)
process_button.pack()

summary_textbox = tk.Text(root, height=10, width=70)
summary_textbox.pack()

copy_button = tk.Button(root, text="Copy to Clipboard", command=lambda: root.clipboard_append(summary_textbox.get("1.0", tk.END)))
copy_button.pack()

root.mainloop()


TclError: ignored